In [ ]:
import sys, time, os, asyncio, warnings
from datetime import datetime
import numpy as np
import pandas as pd
import pickle as pkl
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.backends.backend_pdf import PdfPages
from astropy.time import Time, TimeDelta
from lsst_efd_client import EfdClient
from lsst.summit.utils import dayObsIntToString
from lsst.summit.utils.efdUtils import calcNextDay


In [ ]:
client = EfdClient('usdf_efd')

# First get the data
## Look at successive weekends with no venting

In [ ]:
days = [5,6,7,8,12,13,14,19,21,25,26]
pdf = PdfPages("/home/c/cslage/u/AuxTel/fwhm/Temps_and_Seeing_Mar24.pdf")
fig = plt.figure(figsize=(10,10))

for day in days:
    dayObs = int(f"202403{day:02}")
    nextDayObs = calcNextDay(dayObs)
    dayString = dayObsIntToString(dayObs)
    nextDayString = dayObsIntToString(nextDayObs)

    start = Time(f"{dayString}T08:00:00", scale='utc')
    startObserving = Time(f"{nextDayString}T00:00:00", scale='utc')
    end = Time(f"{nextDayString}T09:00:00", scale='utc')
    endObserving = Time(f"{nextDayString}T08:00:00", scale='utc')
    
    m1Temp = await client.select_time_series('lsst.sal.ESS.temperature', \
                                                    ['temperatureItem5', 'sensorName'],  start, end, index=201)
    m1Temp = m1Temp[m1Temp['sensorName'] == 'AuxTel-ESS03']                                        
    
    m2Temp = await client.select_time_series('lsst.sal.ESS.temperature', \
                                                    ['temperatureItem3', 'sensorName'],  start, end, index=201)
    m2Temp = m2Temp[m2Temp['sensorName'] == 'AuxTel-ESS01']          
    
    outTemp = await client.select_time_series('lsst.sal.ESS.temperature', \
                                                    ['temperatureItem0'],  start, end, index=301)
    
    fanAnemom = await client.select_time_series('lsst.sal.ESS.airFlow', \
                                                    ['speed'],  start, end, index=204)
    def fanOn(speed):
        if speed < 0.40:
            return 0.0
        else:
            return 1.0
    
    fanAnemom['fanOn'] = fanAnemom['speed'].apply(fanOn)
    axs = fig.subplots(3,1)
    plt.subplots_adjust(hspace=0.5)
    plt.suptitle(f"Seeing related temperatures {dayObs}")
    axs[0].set_title("M2 temp (C)")
    m2Temp['temperatureItem3'].plot(ax=axs[0], label="M2 Temp")
    outTemp['temperatureItem0'].plot(ax=axs[0], label="Outside Temp")
    axs[0].legend()
    axs[1].set_title("M1 temp (C) - dotted lines are when observing")
    m1Temp['temperatureItem5'].plot(ax=axs[1], label="M1 Temp")
    outTemp['temperatureItem0'].plot(ax=axs[1], label="Outside Temp")
    axs[1].axvline(startObserving.isot, ls='--', color='green')
    axs[1].axvline(endObserving.isot, ls='--', color='red')
    axs[1].legend()
    axs[2].set_title("Vent Status")
    fanAnemom['fanOn'].plot(ax=axs[2])

    pdf.savefig(fig)  # saves the current figure into a pdf page
    plt.clf()
pdf.close()


In [ ]:
%matplotlib inline
fig, axs = plt.subplots(3,1,figsize=(10,10))
